## Import Libraries

In [38]:
import os
import time
import copy


# 데이터 분석 라이브러리
import numpy as np
import pandas as pd

# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt

from PIL import Image

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import timm

from __future__ import print_function, division
from torch.optim import lr_scheduler
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

cudnn.benchmark = True
plt.ion()   # interactive mode

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda', index=0)

## Bring csv file with path & label

In [41]:
traindata_path = pd.read_csv('train_data.csv')
valdata_path = pd.read_csv('val_data.csv')
testdata_path = pd.read_csv('test_data.csv')

In [49]:
print(len(traindata_path))

15126


## Hyper Parameter
Resize([224,224]) - https://www.programcreek.com/python/example/104834/torchvision.transforms.Resize

In [42]:
transform = transforms.Compose([
#         transforms.Resize([224,224]),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
batch_size = 64
num_workers = 2

## Dataset

In [43]:
def readImage(path_data):
    path_to_image = []
    for path in path_data:
        img = Image.open(path)
        path_to_image.append(img)
    return path_to_image

In [52]:
class MyDataset(Dataset):
    def __init__(self, data, transform, train):
        self._repr_indent = 4
        self.data = data
        self.X = readImage(self.data['img_path'])
        self.y = self.data['label']
        self.transform = transform
        self.train = train
        self.classes = list(set(self.y))
    
    def __getitem__(self, idx):
        X, y = self.X[idx], None
#         X, y = self.X[idx], -1

        if self.transform:
            X = self.transform(X)
        if self.train:
            y = self.y[idx]

        return X,y
#         return torch.tensor(X), torch.tensor(y)
#         return X.clone().detach(), y.clone().detach()
    
    def __len__(self):
        return len(self.X)
    
    def __repr__(self):
        '''
        https://github.com/pytorch/vision/blob/master/torchvision/datasets/vision.py
        '''
        head = "My Custom Dataset : Mask Dataset"
        num_data = self._repr_indent*" " + "Number of datapoints: {}".format(self.__len__())
        num_classes = self._repr_indent*" " + "Number of classes: {}".format(len(self.classes))
        return '\n'.join([head, num_data, num_classes])

In [53]:
train_dataset = MyDataset(
    data = traindata_path,
    transform = transform,
    train = True
)
val_dataset = MyDataset(
    data = valdata_path,
    transform = transform,
    train = True
)
test_dataset = MyDataset(
    data = testdata_path,
    transform = transform,
    train = False
)

In [54]:
train_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 15126
    Number of classes: 18

In [55]:
val_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 3774
    Number of classes: 18

In [56]:
test_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 12600
    Number of classes: 1

## DataLoader

In [57]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)

In [58]:
dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

In [59]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7ff1a5375ca0>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7ff1a5375d00>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7ff1a5375e50>}

## Model
model 생성 및 확인 - https://towardsdatascience.com/getting-started-with-pytorch-image-models-timm-a-practitioners-guide-4e77b4bf9055

In [64]:
model = timm.create_model('resnet18', pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 18)

In [67]:
model(torch.randn(64,3,384,512)).shape

torch.Size([64, 18])

In [68]:
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss() 

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 

In [69]:
model.default_cfg

{'url': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
 'num_classes': 1000,
 'input_size': (3, 224, 224),
 'pool_size': (7, 7),
 'crop_pct': 0.875,
 'interpolation': 'bilinear',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'first_conv': 'conv1',
 'classifier': 'fc',
 'architecture': 'resnet18'}

In [70]:
model.get_classifier()

Linear(in_features=512, out_features=18, bias=True)

In [32]:
import torchsummary
# torchsummary.summary(model,(3,224,224))

In [33]:
# model.state_dict()

## Train
transfer learning train model - https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html   

In [71]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [72]:
def train_model(model, loss_fn, optimizer, num_epochs=25):
    since = time.time()
    
#     best_model_wts = copy.deepcopy(model.state_dict()) 
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            running_loss = 0.
            running_acc = 0.
            if phase == "train":
                model.train() 
            elif phase == "val":
                model.eval() 

            for images, labels in dataloaders[phase]:
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() 

                with torch.set_grad_enabled(phase == "train"):
                    logits = model(images)
                    _, preds = torch.max(logits, 1)
#                     print(logits, labels)
                    loss = loss_fn(logits, labels)
#                     print("loss:",loss)

                    if phase == "train":
                        loss.backward() 
                        optimizer.step() 

                running_loss += loss.item() * images.size(0) 
                running_acc += torch.sum(preds == labels.data) 

            epoch_loss = running_loss / len(dataloaders[phase].dataset) 
            epoch_acc = running_acc / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

In [73]:
model = train_model(model, loss_fn, optimizer, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.5647 Acc: 0.8449
val Loss: 0.2921 Acc: 0.9319

Epoch 1/24
----------
train Loss: 1.4361 Acc: 0.5370
val Loss: 1.6443 Acc: 0.4300

Epoch 2/24
----------
train Loss: 1.1042 Acc: 0.6316
val Loss: 1.4970 Acc: 0.5114

Epoch 3/24
----------
train Loss: 0.9273 Acc: 0.6803
val Loss: 1.3884 Acc: 0.5390

Epoch 4/24
----------
train Loss: 0.7809 Acc: 0.7249
val Loss: 1.5005 Acc: 0.5148

Epoch 5/24
----------
train Loss: 0.6494 Acc: 0.7618
val Loss: 1.6205 Acc: 0.5350

Epoch 6/24
----------
train Loss: 0.5301 Acc: 0.8026
val Loss: 2.0546 Acc: 0.3776

Epoch 7/24
----------
train Loss: 0.3939 Acc: 0.8626
val Loss: 2.0586 Acc: 0.4425

Epoch 8/24
----------
train Loss: 0.2949 Acc: 0.9020
val Loss: 2.0493 Acc: 0.4815

Epoch 9/24
----------
train Loss: 0.1894 Acc: 0.9451
val Loss: 2.4969 Acc: 0.4126

Epoch 10/24
----------
train Loss: 0.1286 Acc: 0.9673
val Loss: 2.4678 Acc: 0.4510

Epoch 11/24
----------
train Loss: 0.0975 Acc: 0.9754
val Loss: 2.4267 Acc: 0.5029

Ep

생기는 오류들    
OSError: image file is truncated (5 bytes not processed) ---> 22 for inputs, labels in dataloaders[phase]: - https://stackoverflow.com/questions/12984426/pil-ioerror-image-file-truncated-with-big-images
   
CUDA error: device-side assert triggered ---> 42 print(loss.item(), inputs.size(0)) - 기존에 내가 test data로 validation을 진행하고 있었음. 여기서 문제가 되는 것은, eval data는 Label이 전부 -1로 선언되어있기 때문에, Cross Entropy Loss를 구할 수가 없었음. 따라서 최초에 EDA를 생성함에 있어서 train, validation, test data를 다시 따로 만들고 그 과정에서도 각 label별로 분할, 그리고 5개의 값이 하나의 폴더 안에 있으면 해당 데이터는 train과 val로 분산되도록 제작
